In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from xlwt import Workbook
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize Chrome webdriver
driver = webdriver.Chrome()

# Open the initial URL




# Initialize a Workbook to save the data
wb = Workbook()
ws = wb.add_sheet('data')
ws.write(0, 0, 'Links')
row = 1

# Create a set to store unique links
unique_links = set()

for i in range(1,67):
    driver.get(f'https://www.datacenters.com/locations?page={i}&per_page=40&query=&withProducts=false&showHidden=false&radius=0&bounds=&circleBounds=&polygonPath=')
    time.sleep(3)
    # Wait for the cookie pop-up and click the accept button
    try:
        cookie_accept_xpath = '//*[@id="onetrust-accept-btn-handler"]'
        cookie_accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, cookie_accept_xpath)))
        cookie_accept_button.click()
    except:
        pass
    
    try:
        driver.execute_script("document.body.style.zoom='50%'")
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        links = soup.find_all(class_="LocationsSearch__anchor__LzIa2")
        
        for link in links:
            link_text = link['href']  # Assuming 'href' contains the link
            if link_text not in unique_links:
                ws.write(row, 0, link_text)
                row += 1
                unique_links.add(link_text)
                print(link_text)
    except:
        pass
    

# Save the Excel file
wb.save('links_output.xls')

# Quit the driver
driver.quit()


Extracting info from each link

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
excel_file = 'links_output.xls'
df = pd.read_excel(excel_file)
urls = df['Links']

# Loop through URLs and make requests
for url in urls:
    try:
        response = requests.get(f'https://www.datacenters.com/{url}')
    except:
        pass
    
    try:
        soup=BeautifulSoup(response.content,'html.parser')
        title=soup.find(class_='LocationProviderDetail__locationName__Cofdh')
        print(title.text)        
        contact=soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[0]
        total_space=soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[1]
        colocation_space=soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[2]
        total_power=soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[3]
        products=soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[4]
        nearest_airport=soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[5]
        print(contact.text)
        print(total_space.text)
        print(colocation_space.text)
        print(total_power.text)
        print(products.text)
        print(nearest_airport.text)
        certification_div=soup.find(class_='LocationProviderDetail__certificationsContainer__Nnx7C')
        cr = ''
        certi = certification_div.find_all('svg', attrs={'aria-labelledby': "icon-title-Ok"})
        
        for cer in certi:
            if cer.text:
                next_span = cer.find_next('span')
                cr = cr+next_span.text+","
        print(cr)
        fc=""
        facility_div=soup.find(class_='LocationProviderDetail__facilityTypesContainer__IzCyu')
        facility=facility_div.find_all('svg', attrs={'aria-labelledby': "icon-title-Ok"})
        for fac in facility:
            if fac.text:
                f_next_span=fac.find_next('span')
                fc=fc+f_next_span.text+","
        print(f'Facility: {fc}')
        
    
    except:
        pass
    break
    

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from xlwt import Workbook

# Load the Excel file and read the URLs
excel_file = 'links_output.xls'
df = pd.read_excel(excel_file)
urls = df['Links']

# Initialize a Workbook to save the data
wb = Workbook()
ws = wb.add_sheet('data')
ws.write(0, 0, 'Title')
ws.write(0, 1, 'Contact')
ws.write(0, 2, 'Total Space')
ws.write(0, 3, 'Colocation Space')
ws.write(0, 4, 'Total Power')
ws.write(0, 5, 'Products')
ws.write(0, 6, 'Nearest Airport')
ws.write(0, 7, 'Certifications')
ws.write(0, 8, 'Facility Types')
row = 1
output_excel_file = 'data_center_output.xls'
# Loop through URLs and make requests
for url in urls:
    try:
        response = requests.get(f'https://www.datacenters.com/{url}')
    except:
        pass
    
    try:
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find(class_='LocationProviderDetail__locationName__Cofdh')
        contact = soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[0]
        total_space = soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[1]
        colocation_space = soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[2]
        total_power = soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[3]
        products = soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[4]
        nearest_airport = soup.find_all(class_='LocationProviderDetail__providerInfoItem__kuPAs')[5]
        certification_div = soup.find(class_='LocationProviderDetail__certificationsContainer__Nnx7C')
        facility_div = soup.find(class_='LocationProviderDetail__facilityTypesContainer__IzCyu')
        
        # Extract certifications
        cr = ''
        certi = certification_div.find_all('svg', attrs={'aria-labelledby': "icon-title-Ok"})
        for cer in certi:
            if cer.text:
                next_span = cer.find_next('span')
                cr = cr + next_span.text + ","
        
        # Extract facility types
        fc = ""
        facility = facility_div.find_all('svg', attrs={'aria-labelledby': "icon-title-Ok"})
        for fac in facility:
            if fac.text:
                f_next_span = fac.find_next('span')
                fc = fc + f_next_span.text + ","
        
        # Write the extracted data to Excel
        ws.write(row, 0, title.text)
        ws.write(row, 1, contact.text)
        ws.write(row, 2, total_space.text)
        ws.write(row, 3, colocation_space.text)
        ws.write(row, 4, total_power.text)
        ws.write(row, 5, products.text)
        ws.write(row, 6, nearest_airport.text)
        ws.write(row, 7, cr)
        ws.write(row, 8, fc)
        print(contact.text)
        print(total_space.text)
        print(colocation_space.text)
        print(total_power.text)
        print(products.text)
        print(nearest_airport.text)
        print(cr)
        
        print(f'Facility: {fc}')
        
    except:
        pass
    row += 1
    wb.save(output_excel_file)
    



print("Data extraction and saving completed.")
